In [614]:
# Learning Resources:
#  - https://www.pythonguis.com/tutorials/tkinter-basic-widgets/
#  - https://www.tcl-lang.org/man/tcl8.6/TkCmd/contents.htm

In [615]:
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

In [616]:
class guiWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.frames={}
        self.frame_count = 0
        self.nested_frame_count = 0
        self.stock_data = None

        # Additional configuration
        self.root.title("stockr")
        self.root.minsize(255,330)
        self.root.maxsize(2550,3300)
        self.root.geometry("1200x800")
        self.root.configure(background='lightgrey')


In [617]:
class guiVisuals:
    def __init__(self, guiWindow, type, target_frame=None):
        self.guiWindow = guiWindow
        self.type = type.lower()
        if target_frame is not None and target_frame in guiWindow.frames:
            self.target_frame = guiWindow.frames[target_frame]
        else:
            self.target_frame = guiWindow.root
        self.stock_data = guiWindow.stock_data
        self.dispatch_type()
    
    def dispatch_type(self):
        # Redirect type parameter to visual call
        method_name = f"create_{self.type}"
        method = getattr(self, method_name, None)
        if method:
            method()
        else:
            print(f"[Warning]: Visuals visual type '{self.type}' not yet implemented.")

    def create_label(self):
        # Create a label
        self.label = tk.Label(self.guiWindow.root, text="testing label!", font=('Helvetica', 19))
        self.label.pack()
        return self.label
        # Return to calibrate positioning!

    def create_table(self):
        ticker = self.stock_data["tickers"][0]
        df = self.stock_data[f"{ticker}_data"].copy()
        df.reset_index(inplace=True)

        style = ttk.Style()
        style.theme_use("default")

        style.configure("Treeview",
                        font=("Helvetica", 10),
                        rowheight=22,
                        background="#f8f9fa",
                        fieldbackground="#f8f9fa",
                        foreground="black")

        style.configure("Treeview.Heading",
                        font=("Helvetica", 10, "bold"),
                        background="#dee2e6",
                        foreground="black")

        style.map("Treeview", background=[("selected", "#cce5ff")])
        style.configure("Treeview.Row", padding=1)

        tree = ttk.Treeview(self.target_frame)
        tree["columns"] = list(df.columns)
        tree["show"] = "headings"

        tree.tag_configure("evenrow", background="#ffffff")
        tree.tag_configure("oddrow", background="#e9ecef")

        for col in df.columns:
            width = max(80, min(150, int(df[col].astype(str).map(len).mean() * 7)))
            tree.column(col, anchor="center", width=width)
            tree.heading(col, text=col)

        for i, (_, row) in enumerate(df.iterrows()):
            tag = "evenrow" if i % 2 == 0 else "oddrow"
            tree.insert("", "end", values=list(row), tags=(tag,))

        tree.pack(fill='both', expand=True)
        

In [618]:
class guiInteractables:
    def __init__(self, guiWindow, type):
        self.guiWindow = guiWindow
        self.type = type.lower()
        self.dispatch_type()

    def dispatch_type(self):
        # Redirect type parameter to visual call
        method_name = f"create_{self.type}"
        method = getattr(self, method_name, None)
        if method:
            method()
        else:
            print(f"[Warning]: Interactable visual type '{self.type}' not yet implemented.")
    
    def create_combobox(self):
        # Create a combobox
        self.combobox = ttk.Combobox(self.guiWindow.root, values=["Option 1", "Option 2", "Option 3"])
        self.combobox.set("Set Default Here")
        self.combobox.pack()
        return self.combobox
        # Going to want additional functionality here, calbirating size of comboboxes to max size of all boxes

    def create_entry(self):
        # Create an entry field
        self.entry = tk.Entry(self.guiWindow.root, width=30)
        self.entry.insert(0, "Enter text here")
        self.entry.pack()
        return self.entry
        # Here will want to use bind() function to display error messages if input is invalid - maybe.
    
    def create_radiobuttons(self):
        # Create radio buttons
        self.var = tk.StringVar(value="Option 1")
        self.radio1 = tk.Radiobutton(self.guiWindow.root, text="Option 1", variable=self.var, value="Option 1")
        self.radio1.pack()
        return (self.radio1)

In [619]:
class guiFrames:
    def __init__(self, guiWindow, width=800, height=600, parent_frame=None,
                 use_place=False, relx=None, rely=None, anchor=None, x=None, y=None,
                 padx=None, pady=None, side=None, fill=None):
        self.guiWindow = guiWindow
        self.width = width
        self.height = height
        self.parent_frame = parent_frame
        self.use_place = use_place

        # Placement parameters
        self.relx = relx
        self.rely = rely
        self.anchor = anchor
        self.x = x # padding for parent frames
        self.y = y # padding for parent frames
        self.side = side
        self.fill = fill

        self.name = None
        self.create_frame()

    def create_frame(self):
        parent = self.guiWindow.frames[self.parent_frame] if self.parent_frame in self.guiWindow.frames else self.guiWindow.root

        if parent == self.guiWindow.root:
            self.frame = tk.Frame(parent, bg='white', width=self.width, height=self.height)
        else:
            self.frame = tk.Frame(parent, bg='red', width=self.width, height=self.height)
        self.frame.pack_propagate(False)

        if self.use_place and self.relx is not None and self.rely is not None:
            self.frame.place(
                relx=self.relx, rely=self.rely, anchor=self.anchor, x=self.x, y=self.y,)
        else:
            # Default to `pack` layout (call this an error)
            self.frame.pack(anchor="center")

        # Register frame
        if parent == self.guiWindow.root:
            self.guiWindow.frame_count += 1
            self.name = f"frame_{self.guiWindow.frame_count}"
        else:
            self.guiWindow.nested_frame_count += 1
            self.name = f"nested_frame_{self.guiWindow.nested_frame_count}"

        self.guiWindow.frames[self.name] = self.frame
        return self.frame

In [620]:
class dataOperations:
    def __init__(self, tickers, start_date, end_date, interval="60m"):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.stock_data = {}

    def fetch_stock_data(self):
        """
        Note: plucked from old project, modified as needed. Fetch stock data for given tickers and date range.
        
        Args:
            tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
            start_date (str): Start date (e.g., '2024-10-01').
            end_date (str): End date (e.g., '2024-10-31').
            interval (str): Interval for data download, recommended entries: 30m, 60m, 1d, 1wk, 1mo.
        
        Returns:
            dict: Dictionary with ticker keys and their data as values, and a list of tickers.
        """
        for ticker in self.tickers:
            # Download data for each ticker
            data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
            # data = data.drop(columns=["Close", "Volume"], errors="ignore")
            self.stock_data[f"{ticker}_data"] = data
            self.stock_data["tickers"] = self.tickers
        
        guiWindow.stock_data = self.stock_data  # Update the stock_data in guiWindow
        return self.stock_data

    def correct_stock_data(self, stock_data):
        """
        Correct stock data by removing multi-index levels and renaming index column.
        
        Args:
            stock_data (dict): Dictionary of stock data with ticker keys.
        """
        for ticker_key, dataframe in stock_data.items():
            if ticker_key == "tickers":
                continue
            dataframe = dataframe.droplevel("Ticker", axis=1)
            dataframe.reset_index(inplace=True)
            dataframe.columns.name = "Data Index"
            if "Datetime" in dataframe.columns:
                dataframe["Datetime"] = pd.to_datetime(dataframe["Datetime"]).dt.strftime('%Y-%m-%d %H:%M')
            stock_data[ticker_key] = dataframe

        guiWindow.stock_data = stock_data  # Update the stock_data in guiWindow
        return stock_data

    def format_large_number(value):
        """Helper function to format large numbers for display."""
        if value is None:
            return None
        elif value >= 1e9:
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:
            return f"{value / 1e6:.2f}M"
        elif value >= 1e3:
            return f"{value / 1e3:.2f}K"
        else:
            return f"{value:.2f}"

In [621]:
# Define data request
data_op = dataOperations(
    tickers=["AAPL", "MSFT",],
    start_date="2025-06-01",
    end_date="2025-06-30",
    interval="1d")

In [622]:
# Fetch and assign data
window = guiWindow()
window.stock_data = data_op.fetch_stock_data()
window.stock_data = data_op.correct_stock_data(window.stock_data)

# GUI Setup
# guiVisuals(window, "label")
# guiInteractables(window, "combobox")
# guiInteractables(window, "radiobuttons")
# guiInteractables(window, "entry")
main_frame = guiFrames(window, width=875, height=500, use_place=True, relx=1, rely=0, anchor='ne', x=-20, y=20)
table_frame = guiFrames(
    window, width=865, height=490, parent_frame=main_frame.name, use_place=True, anchor='center',relx=0.5, rely=0.5)
guiVisuals(window, "table", target_frame=table_frame.name)

main_toggle_frame = guiFrames(window, width=250, height=500, use_place=True, relx=0, rely=0, anchor='nw', x=20, y=20)

# Run main loop
window.root.mainloop()

/var/folders/yt/068k7p4x3934d27j929_47ww0000gn/T/ipykernel_38566/446886656.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
[*********************100%***********************]  1 of 1 completed
/var/folders/yt/068k7p4x3934d27j929_47ww0000gn/T/ipykernel_38566/446886656.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=self.start_date, end=self.end_date, interval=self.interval, rounding=True)
[*********************100%***********************]  1 of 1 completed
